# CoCoA :
#  Application to SVM
# Local SDCA

# Philippe Peroumal ENSAE

In [98]:
import numpy as np
%pylab inline
import math
import concurrent.futures

Populating the interactive namespace from numpy and matplotlib


/Users/philippe/anaconda/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['e', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Local Sdca

In [79]:
def localSDCA(alpha_input,w_input,A_data_k,b_k,n_samples,H) :
    
    n_features,n_k=A_data_k.shape
    # initialiser
    # w_0
    # deta_alpha_k=0
    w=w_input
    d_a=np.zeros(n_k)
    
    
    delta_incr=np.zeros(n_k)
    alpha_k=alpha_input
    for h in range(H):
        # choose i in range( n_k)       
        id_x=np.random.randint(0, n_k)  # id_x=i
        lambda_p=1  # valeur par defaut
        y=b_k[id_x]
        
        def l_star(u):
            if -1<= y*u <= 0:
                res= y*u
            else :
                res=math.inf
            return res
        
        def l_star_grad(u):
            if -1<= y*u <= 0:
                    res= y
            else     :
                res=math.inf
                
            lambdda=1
            
           
            
        # We have a closed form for the solution http://www.vlfeat.org/api/svm-sdca.html
        alpha_old=alpha_k[id_x]
        delta= y* max(0,min (1,y*(delta_incr[id_x]+ alpha_old  )    )) -alpha_old
            
        # update alpha_i(h)
        alpha_k[id_x]= alpha_k[id_x]+delta
        
        # update delta alpha k
        delta_incr[id_x]=delta_incr[id_x]+delta
        
        #update w_h
        w=w+1/(lambdda*n_samples)*delta_incr.dot(A_data_k[::,id_x])
        
    delta_w=A_data_k.dot(delta_incr)   
            
    return  delta_incr,delta_w  
    

In [95]:
def localSDCA_bis(alpha_input,w_input,A_data_k,b_k,n_samples,H):
    delta_incr,delta_w=localSDCA(alpha_input,w_input,A_data_k,b_k,n_samples,H) 
    alpha_input=alpha_input+1/K*delta_incr
    return alpha_input,delta_w

In [129]:
def Cocoa(K,A_data,b,nb_iter,H) :
    
    n_features,n_samples=A_data.shape
    
    # split les données
    A_data_split=np.array_split(A_data,K) 
    y_split=np.array_split(b,K)
    n_k=[len(y_split[k]) for k in range(len(y_split))  ]
    delta_w_vec= np.zeros(K)
    #paramatères des SDCA locales
    alpha_input_vec=[np.zeros(n_k) for j in n_k]    
    delta_w=np.zeros(n_features)
    
    for iter in range(nb_iter) :
        futures = []
        with concurrent.futures.ProcessPoolExecutor() as executor:
            for k in range(K) :
                futures.append(executor.submit(localSDCA_bis,alpha_input_vec[k],w_input,A_data_split[k],b_k,n_samples,H  ))
            concurrent.futures.wait(futures)
            # attente des workers
            
            for k in range(K) :
                alpha_input[k],delta_w[K]= futures[k].result()
            # on recupère leurs resultats
            
            # on réduit
            w_input= w_input+ 1/K*delta_w.sum()    

# A toy dataset

In [123]:
n_samples, n_features = 100, 10
A_data = np.random.randn(n_features,n_samples)
w = np.random.randn(n_samples)
b = np.sign(A.dot(w) + np.random.randn(n_features))

In [ ]:
nb_iter=3
H=3
Cocoa(K,A_data,b,nb_iter,H)

# A real data set

In [110]:
import os, hashlib,urllib

# load a large, sparse dataset such as RCV1
def load_rcv1(md5_check=True):
    from os.path import expanduser
    home = expanduser("~")
    dir_name = os.path.join(home, 'copt_data')
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    file_path = os.path.join(dir_name, 'rcv1_full.binary.bz2')
    if not os.path.exists(file_path):
        print('RCV1 dataset is not present in data folder. Downloading it ...')
        url = 'http://s3-eu-west-1.amazonaws.com/copt.bianp.net/datasets/rcv1_full.binary.bz2'
        urllib.request.urlretrieve(url, file_path)
        print('Finished downloading')
    if md5_check:
        h = hashlib.md5(open(file_path, 'rb').read()).hexdigest()
        if not h == '6131cf16515e9cce08d112c880b6b817':
            print('MD5 hash do not coincide')
            print('Removing file and re-downloading')
            os.remove(file_path)
            return load_rcv1()
    from sklearn import datasets
    return datasets.load_svmlight_file(file_path)

In [132]:
A_data, b = load_rcv1()

In [125]:
A_data.shape

(10, 100)

In [115]:
nb_iter=3
H=3